<a href="https://colab.research.google.com/github/ranamahmoud12/AssignmentDL/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
import keras
import itertools
from sklearn.utils import shuffle
from keras.metrics import AUC, Recall, Precision, SpecificityAtSensitivity, SensitivityAtSpecificity, FalseNegatives, FalsePositives, TrueNegatives, TruePositives,F1Score
from keras.optimizers.schedules import ExponentialDecay

from keras import layers
from keras import models
from keras.applications.resnet50 import ResNet50
# from keras.applications.vgg19 import VGG19

from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

from keras.applications.vgg19 import preprocess_input
from keras.models import Sequential
from sklearn.utils.class_weight import compute_class_weight

from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import os
from keras.regularizers import l2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob as gb
import cv2

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [8]:
batch_size = 32
img_width=224
img_height=224
img_size=128
epochs=50
NUM_CLASSES=2

In [9]:
data_generator = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    validation_split=0.2
)

train_generator = data_generator.flow_from_directory(
    '/content/drive/MyDrive/DeepLearning/Train',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='binary',  # Change this to 'binary'
    seed=42,
    subset='training'
)

validation_generator = data_generator.flow_from_directory(
    '/content/drive/MyDrive/DeepLearning/Test',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    shuffle=True,
    class_mode='binary',  # Change this to 'binary'
    seed=42,
    subset='validation'
)

Found 84 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [10]:
size = []

# Loop through folders in the training directory
for folder in os.listdir('/content/drive/MyDrive/DeepLearning/Train'):
    folder_path = os.path.join('/content/drive/MyDrive/DeepLearning/Train', folder)

    # Check if it's a directory
    if os.path.isdir(folder_path):
        # Use glob to get all JPEG files in the folder
        files = gb.glob(os.path.join(folder_path, '*.jpg'))

        # Loop through each file
        for file in files:
            # Read the image
            image = plt.imread(file)
            # Get the size and append to the list
            size.append(image.shape)

# Create a Pandas Series and print the value counts
size_series = pd.Series(size)
print(size_series.value_counts())

(256, 256, 3)    104
dtype: int64


In [11]:
size = []

# Loop through folders in the test directory
for folder in os.listdir('/content/drive/MyDrive/DeepLearning/Test'):
    folder_path = os.path.join('/content/drive/MyDrive/DeepLearning/Test', folder)

    # Check if it's a directory
    if os.path.isdir(folder_path):
        # Use glob to get all JPEG files in the folder
        files = gb.glob(os.path.join(folder_path, '*.jpg'))

        # Loop through each file
        for file in files:
            # Read the image
            image = plt.imread(file)
            # Get the size and append to the list
            size.append(image.shape)

# Create a Pandas Series and print the value counts
size_series = pd.Series(size)
print(size_series.value_counts())

(256, 256, 3)    24
dtype: int64


In [12]:
# Create a mapping dictionary for class labels
class_mapping = {'Class (1)': 0, 'Class (2)': 1}  # Add more classes as needed
img_size=256

In [13]:
def load_images(file_paths, img_size):
    images = []
    for file_path in file_paths:
        # Read and preprocess the image
        img = cv2.imread(file_path)
        img = cv2.resize(img, (img_size, img_size))
        img = img / 255.0  # Normalize the pixel values to the range [0, 1]
        images.append(img)
    return np.array(images)

# Now, your code to load and preprocess training images
x_train = []
y_train = []

# Loop through folders in the training directory
for folder in os.listdir('/content/drive/MyDrive/DeepLearning/Train'):
    folder_path = os.path.join('/content/drive/MyDrive/DeepLearning/Train', folder)

    if os.path.isdir(folder_path):
        files = gb.glob(os.path.join(folder_path, '*.jpg'))
        x_train.extend(files)
        y_train.extend([folder] * len(files))

# Convert to NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

# Load and preprocess training images
x_train = load_images(x_train, img_size)
y_train = np.array([class_mapping[label] for label in y_train])

print(x_train.shape)
print(y_train.shape)

(104, 256, 256, 3)
(104,)


In [14]:
x_test = []
y_test = []

# Loop through folders in the test directory
for folder in os.listdir('/content/drive/MyDrive/DeepLearning/Test'):
    folder_path = os.path.join('/content/drive/MyDrive/DeepLearning/Test', folder)

    # Check if it's a directory
    if os.path.isdir(folder_path):
        # Use glob to get all JPEG files in the folder
        files = gb.glob(os.path.join(folder_path, '*.jpg'))

        # Append file paths and corresponding labels
        x_test.extend(files)
        y_test.extend([folder] * len(files))

# Convert to NumPy arrays
x_test = np.array(x_test)
y_test = np.array(y_test)

# Load and preprocess test images
x_test = load_images(x_test, img_size)
y_test = to_categorical(np.array([class_mapping[label] for label in y_test]))

print(x_test.shape)
print(y_test.shape)

(24, 256, 256, 3)
(24, 2)


In [15]:
x_train=x_train/255
x_test=x_test/255

In [16]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=42)

print(x_valid.shape)
print(y_valid.shape)

(32, 256, 256, 3)
(32,)


In [17]:

input_shape = (img_size, img_size, 3)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

model = Sequential()

# Add the pre-trained AlexNet model
model.add(base_model)

# Flatten the output of the AlexNet model
model.add(Flatten())

# Add some dense layers with dropout for classification
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.summary()



94765736/94765736 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 8, 8, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 4096)              536875008 
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dropout_1 (Dropout)         (None, 4096)              0         
                                                         

In [18]:
class_counts = np.array([10, 20, 30])
total_samples = np.sum(class_counts)
class_weights = total_samples / (len(class_counts) * np.array(class_counts, dtype=float))

class_weights_list = class_weights.tolist()
class_weight_dict = {i: class_weights_list[i] for i in range(len(class_weights_list))}

print(class_weight_dict)

{0: 2.0, 1: 1.0, 2: 0.6666666666666666}


In [ ]:

# Compile the model with the specified class weights
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        AUC(),
        Recall(),
        Precision(),
        SpecificityAtSensitivity(0.5),
        SensitivityAtSpecificity(0.5),
        FalseNegatives(),
        FalsePositives(),
        TrueNegatives(),
        TruePositives()
    ]
)

# Train the model with class weights
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_valid, y_valid), callbacks=[early_stopping], class_weight=class_weight_dict)

Epoch 1/50


In [ ]:
# Ensure test data is in the correct format
x_test = x_test.astype(np.float32)

# Evaluate the model on the test set
evaluation_results = model.evaluate(x_test, y_test)
print(f"Test Loss: {evaluation_results[0]}")
print(f"Test Accuracy: {evaluation_results[1]}")


In [ ]:
title = "Val loss for "+dataset+" "+impl_type+"\n"

plt.plot(log_data['loss'])
plt.plot(log_data['val_loss'])
plt.title(title)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'best')
plt.grid(b=True, which='major', axis='both')

img_path = work_dir+'vLoss_'+checkpointer_name[8:-5]+'.png'
plt.savefig(img_path, dpi=600)
plt.show()
print('img_path =', img_path)

In [ ]:
title = "Val acc for "+dataset+" "+impl_type+"\n"

plt.plot(log_data['accuracy'])
plt.plot(log_data['val_accuracy'])
plt.title(title)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy','val_accuracy'], loc = 'best')
plt.grid(b=True, which='major', axis='both')

img_path = work_dir+'vAcc_'+checkpointer_name[8:-5]+'.png'
plt.savefig(img_path, dpi=600)
plt.show()
print('img_path =', img_path)